In [5]:
import ee
import geemap
import os

# Authenticate to Earth Engine
try:
  ee.Initialize(project='envr451-2024')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='envr451-2024')

# Planet
Planet provides us with 4.77m resolution all the way to Jan 2016.
https://developers.google.com/earth-engine/datasets/catalog/projects_planet-nicfi_assets_basemaps_americas

Note - with the NICFI project, we can only get biannual images up until 2020; in 2020 we get
planet_medres_normalized_analytic_2020-06_2020-08_mosaic
planet_medres_normalized_analytic_2020-09_mosaic
planet_medres_normalized_analytic_2020-10_mosaic
planet_medres_normalized_analytic_2020-11_mosaic
planet_medres_normalized_analytic_2020-12_mo

Here, I am considering medians for all years. Consider, however, that some images have clouds, and we expect worse quality for the images from 2015-2020 since 2015-2019 only has 2 images per year, and 2020 has 5, while for 2021-2024 we get all 12 images for the year.

This can be improved once we get all images from McGill. Also, these should be at 3.7m resolution.saic

In [260]:
piura_shp = ee.FeatureCollection('projects/envr451-2024/assets/piura').geometry()
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2015-01-01', '2024-12-31')) \
            .map(lambda image: image.clip(piura_shp))

# Define the years for which you want to calculate geometric medians.
years = ee.List.sequence(2015, 2024)

# Function to calculate geometric median for each year.
def calculate_median(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    yearly_collection = basemap.filterDate(start_date, end_date)
    median = yearly_collection.reduce(ee.Reducer.median())
    return median.set('year', year)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura_shp))
yearly_medians_list = yearly_medians.toList(yearly_medians.size())

# Get the number of images.
num_images = yearly_medians_list.size().getInfo()

# Loop over the list and export each image.
for i in range(num_images):
    year = years.get(i).getInfo()
    task = ee.batch.Export.image.toDrive(
        image=ee.Image(yearly_medians_list.get(i)),
        description = 'Planet_Satellite_5m_median_' + str(year),
        fileNamePrefix = 'Planet_Satellite_5m_median_' + str(year),
        region=piura_shp,
        scale=5,
        maxPixels=1e13
    )
    task.start()

# vis_basemap = {'bands': ['R_median', 'G_median', 'B_median'], 'min': 64, 'max': 5454, 'gamma': 1.8}
# Map = geemap.Map()
# Map.centerObject(piura_shp, 13)
# Map.addLayer(yearly_medians.first(), vis_basemap, 'last_image')
# Map

# Copernicus Land Cover
100m resolution
2015-01-01 to 2019-12-31

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description

In [385]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select('discrete_classification').clip(piura_shp)

palette = [
  '006400', # forest (10)
  'ffbb22', # shrubland (20)
  'ffff4c', # grassland (30)
  'f096ff', # cropland (40)
  'fa0000', # urban (50)
  'b4b4b4', # bare/sparse (60)
  'f0f0f0', # snow (70)
  '0064c8' # water (80)
]

visualization = {
    'min':10, 'max': 80, 'palette': palette,
}
grid = geemap.latlon_grid(
    lat_step=0.01, lon_step=0.01, west=-81.5, east=-80.5, south=-8, north=9
)

m = geemap.Map()
m.center_object(dataset1)
m.add_layer(dataset1, visualization, 'Landcover')
m.add_layer(dataset2, visualization, 'Landcover2')
m.addLayer(grid, {}, "Coordinate Grid")
m

KeyboardInterrupt: 

In [352]:
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2021-04-02')
END = START.advance(1, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(28.6729, 52.4305)),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
# m.set_center(20.6729, 52.4305, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

EEException: Element.get: Parameter 'object' is required.

# Dynamic World
10m land use, from 2015-06-27 to today. Near real time.


https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1#colab-python


# ESA WorldCover
10m resolution
2020-01-01 - 2022-12-31

https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100
https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200

In [330]:
dataset1 = ee.ImageCollection('ESA/WorldCover/v100').map(lambda image: image.clip(piura_shp))
dataset2 = ee.ImageCollection('ESA/WorldCover/v200').map(lambda image: image.clip(piura_shp))

palette = [
  '006400', # forest (10)
  'ffbb22', # shrubland (20)
  'ffff4c', # grassland (30)
  'f096ff', # cropland (40)
  'fa0000', # urban (50)
  'b4b4b4', # bare/sparse (60)
  'f0f0f0', # snow (70)
  '0064c8' # water (80)
]

visualization = {
    'min':10, 'max': 80, 'palette': palette,
}

# m = geemap.Map()
# m.center_object(dataset1)
# m.add_layer(dataset1, visualization, 'Landcover')
# m.add_layer(dataset2, visualization, 'Landcover2')
# m

# ESRI Land Use Land Cover

https://livingatlas.arcgis.com/landcover/

In [341]:
esri_lulc2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic().clip(piura_shp)

dict = {
  "names": [
    "Water", "Trees", "Grass","Flooded Vegetation","Crops","Scrub/Shrub",
    "Built Area","Bare Ground","Snow/Ice","Clouds"
  ],
  "colors": [
    "1A5BAB","358221","A7D282","87D19E","FFDB5C","EECFA8",
    "ED022A","EDE9E4","F2FAFF","C8C8C8"
  ]
}

# Create a palette from the dict object
palette = dict['colors']

# Create a visualization parameter object
visualization = {'min': 1, 'max': len(dict['names']), 'palette': palette}

m = geemap.Map()
m.center_object(esri_lulc2020)
m.add_layer(esri_lulc2020, visualization, 'Landcover2')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# ESRI Topography

All of these should have a finely traced but properly scaled grid with tick marks corresponding to decimal long./lat. degrees.

# NASA Elevation

In [380]:
# Import the dataset and select the elevation band.
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation')

# Set elevation visualization properties.
elevation_vis = {'min': 0, 'max': 2500, 'palette': ['green', 'red']}

# Set elevation <= 0 as transparent.
elevation_masked = elevation.updateMask(elevation.gt(0)).clip(piura_shp)

Map = geemap.Map()
Map.addLayer(elevation_masked, elevation_vis, 'Elevation')
Map.centerObject(piura_shp, 12)
# Map

In [381]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.centerObject(piura_shp)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m.addLayer(elevation_masked, elevation_vis, 'Elevation')
m.add_layer(piura_shp, {})
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [375]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
    
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)
dataset

# visualization = {
#     'min': 0.0,
#     'max': 0.3,
#     'bands': ['B4', 'B3', 'B2'],
# }

# m = geemap.Map()
# m.centerObject(piura_shp)
# m.add_layer(dataset.median(), visualization, 'RGB')
# m

# Hydrology

https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1

In [332]:
dataset = ee.Image('MERIT/Hydro/v1_0_1').clip(piura_shp)

visualization = {'bands': ['viswth']}

Map.centerObject(piura_shp, 12)

Map.addLayer(dataset, visualization, 'River width')
